In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/human-faces/Humans/1 (2916).jpg
/kaggle/input/human-faces/Humans/1 (607).jpg
/kaggle/input/human-faces/Humans/1 (3767).jpg
/kaggle/input/human-faces/Humans/1 (576).jpg
/kaggle/input/human-faces/Humans/1 (1856).jpg
/kaggle/input/human-faces/Humans/1 (1464).jpg
/kaggle/input/human-faces/Humans/1 (1290).jpg
/kaggle/input/human-faces/Humans/1 (1341).jpg
/kaggle/input/human-faces/Humans/1 (2598).jpg
/kaggle/input/human-faces/Humans/1 (789).jpg
/kaggle/input/human-faces/Humans/1 (882).jpg
/kaggle/input/human-faces/Humans/1 (5469).jpg
/kaggle/input/human-faces/Humans/1 (6466).jpg
/kaggle/input/human-faces/Humans/1 (6835).jpg
/kaggle/input/human-faces/Humans/1 (6027).jpg
/kaggle/input/human-faces/Humans/1 (4288).jpg
/kaggle/input/human-faces/Humans/1 (550).jpg
/kaggle/input/human-faces/Humans/1 (509).jpg
/kaggle/input/human-faces/Humans/1 (2135).jpg
/kaggle/input/human-faces/Humans/1 (3690).jpg
/kaggle/input/human-faces/Humans/1 (3664).jpg
/kaggle/input/human-faces/Humans/1 (6160

In [28]:
!pip install torch-xla -f https://storage.googleapis.com/pytorch-tpu-releases/wheels/tpuvm/torch_xla.html

Looking in links: https://storage.googleapis.com/pytorch-tpu-releases/wheels/tpuvm/torch_xla.html
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 MB 17.3 MB/s eta 0:00:00:00:0100:01


# **Importing Libraries**

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tqdm import tqdm
from PIL import Image
import os

# **Denoising Encoder**

In [2]:
class DenoisingAutoencoder(nn.Module):
    def __init__(self):
        super(DenoisingAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2, padding=0),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 3, kernel_size=2, stride=2, padding=0),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


In [3]:
class FaceDataset(Dataset):
    def __init__(self, image_folder, transform=None):
        self.image_folder = image_folder
        self.transform = transform
        self.image_paths = [os.path.join(image_folder, img) for img in os.listdir(image_folder)]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image


In [4]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # 
    transforms.ToTensor(),
])


In [5]:
dataset = FaceDataset("/kaggle/input/human-faces/Humans", transform=transform)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)


# **Initializing Model, Loss Function and Optimizer**

In [6]:
model = DenoisingAutoencoder().to("cuda")
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=5e-4)


# **Training**

In [11]:
num_epochs = 5

for epoch in range(num_epochs):
    epoch_loss = 0  
    for images in tqdm(dataloader):
        images = images.to('cuda')
        noisy_images = images + 0.1 * torch.randn_like(images).to('cuda')

        outputs = model(noisy_images)
        loss = criterion(outputs, images)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs} completed. Average Loss: {epoch_loss / len(dataloader)}")

 17%|█▋        | 149/903 [00:43<03:15,  3.85it/s]/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
100%|██████████| 903/903 [04:32<00:00,  3.31it/s]


Epoch 1/5 completed. Average Loss: 0.00665663723737942


100%|██████████| 903/903 [03:21<00:00,  4.48it/s]


Epoch 2/5 completed. Average Loss: 0.0021543651736719927


100%|██████████| 903/903 [03:23<00:00,  4.44it/s]


Epoch 3/5 completed. Average Loss: 0.0018631937143424834


100%|██████████| 903/903 [03:20<00:00,  4.50it/s]


Epoch 4/5 completed. Average Loss: 0.0016833050367303795


100%|██████████| 903/903 [03:21<00:00,  4.49it/s]

Epoch 5/5 completed. Average Loss: 0.001571556375076291


In [12]:
torch.save(model.state_dict(), "denoising-autoencoder-model.pth")

# Evaluation

In [13]:
# Initialize the model
model = DenoisingAutoencoder()

# Load the saved weights
model.load_state_dict(torch.load("/kaggle/working/denoising-autoencoder-model.pth"))

# Set the model to evaluation mode
model.eval()


<ipython-input-13-e53d55d6e92a>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/kaggle/working/denoising-autoencoder-model.pth"))


DenoisingAutoencoder(
  (encoder): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (decoder): Sequential(
    (0): ConvTranspose2d(128, 64, kernel_size=(2, 2), stride=(2, 2))
    (1): ReLU()
    (2): ConvTranspose2d(64, 3, kernel_size=(2, 2), stride=(2, 2))
    (3): Sigmoid()
  )
)

In [14]:
# Load a test image
test_image_path = "/kaggle/input/chopper/Screenshot 2024-10-03 at 10.39.45PM.png"
test_image = Image.open(test_image_path).convert("RGB")

# Apply transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])
test_image_tensor = transform(test_image).unsqueeze(0)  # Add batch dimension

# Add noise to the image
noisy_image_tensor = test_image_tensor + 0.1 * torch.randn_like(test_image_tensor)
noisy_image_tensor = torch.clamp(noisy_image_tensor, 0.0, 1.0)  # Ensure pixel values are in [0, 1]
